In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from helperFunctions.Helpers import Helper
import chromadb
import ollama
import pypdf
import faiss
import asyncio
from tqdm.asyncio import tqdm
from tqdm import tqdm
from langchain_community.vectorstores import FAISS
import numpy as np
from multiprocessing import Pool
import pinecone
import os
from dotenv import load_dotenv




In [7]:
!pip install pinecone-client


In [8]:
pip show pinecone-client


Name: pinecone-client
Version: 5.0.1
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: c:\users\surya\anaconda3\lib\site-packages
Requires: certifi, pinecone-plugin-inference, pinecone-plugin-interface, tqdm, typing-extensions, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [9]:
# # restart kernal
# import IPython
# IPython.Application.instance().kernel.do_shutdown(True)


In [10]:
h=Helper()


In [11]:
vidhuraneethi_pdf_text=h.extract_text_from_pdf(r"D:\Surya files\GenerativeAI\DocuChatProject\pdfDocuments\Vidura Niti Sanskrit Text- With English Translation - K.M.Ganguli.pdf")
yakshaprasna_pdf_text=h.extract_text_from_pdf(r"D:\Surya files\GenerativeAI\DocuChatProject\pdfDocuments\yakshaprashna.pdf")


In [12]:
vidhuraneethi_pdf_text

' \n \nVidura Niti  \n॥  ॥ \n \n \n \n \n \n \nSource of E -texts:  \n \nSanskrit UnicodeText: www.sanskritdocuments.org  \nSanskrit Text of Mahabharata: Vedic Reserve at \nwww.mum.org  \nEnglish Translation by KM Ganguli:  \nwww.sacred -texts.com  Vidura Niti  \n2 ॥अथ  \n॥ \n(Vidura’s teachings on right conduct \nextracted from Mahabharata)  \n \n। \n  \n॥ १॥ \n \n"Vaisampayana said, \'King Dhritarashtra \nendued with great wisdom (then) said to the \norderly -in-waiting, \'I desire to see Vidura. \nBring him here without  delay.\'  \n \n।  \n॥ २॥ \n \nDespatched by Dhritarashtra, the messenger \nwent to Kshatri and said, \'O thou of great Vidura Niti  \n3 wisdom, our lord the mighty king desireth to \nsee thee.\'  \n \n।  \n॥ ३॥ \n \nThus addressed, Vidura (set out and) coming \nto the palace, spoke unto the orderly, \n\'Apprise Dhritarashtra of my arrival.\'  \n \n। \n।  \n॥ ४॥ \n \nThereupon the orderly went to \nDhritarashtra, and said, O, foremost of kings, \nVidura is here at th

In [13]:
yakshaprasna_pdf_text

'Yaksha Prashna\nयǘ ĲŬन\nThe Story of the Righteous Crane\nMahabharata, Aranya Parva , 311–12\nVaisampayana said, ‘Yudhishthira saw his brothers, each possessed of the glory of\nIndra himself, lying dead like the Regents of the world dropped from their spheres\nat the end of the Yuga. And beholding Arjuna lying dead, with his bow and arrows\ndropped on the ground, and also Bhimasena and the twins motionless and deprived\nof life, the king breathed a hot and long sigh, and was bathed in tears of grief.\nAnd beholding his brothers lying dead, the mighty armed son of Dharma with heart\nracked in anxiety, began to lament profusely, saying, “Thou hadst, O mighty-armed\nVrikodara, vowed, saying,— I shall with mace smash the thighs of Duryodhana in battle!\nO enhancer of the glory of the Kurus, in thy death, O mighty-armed and high-souled\none, all that hath become fruitless now! The promises of men may be ineffectual;\nbut why have the words of the gods uttered in respect of thee been thus f

In [14]:
# Calculate chunk size dynamically based on the text in pdf_1_text and pdf_2_text
avg_chunk_size_pdf_1 = h.calculate_avg_chunk_size(vidhuraneethi_pdf_text)
avg_chunk_size_pdf_2 = h.calculate_avg_chunk_size(yakshaprasna_pdf_text)

In [15]:
print("avg_chunk_size_pdf_1",avg_chunk_size_pdf_1)
print("avg_chunk_size_pdf_2",avg_chunk_size_pdf_2)

avg_chunk_size_pdf_1 23.84596631205674
avg_chunk_size_pdf_2 69.63106796116504


## Split the text data in to chunks using recurcide splitter

In [16]:
# Based on the average chunk sizes, set the chunk_size and chunk_overlap
# For example, if avg_chunk_size is ~300 characters, you can use a chunk size of ~300
chunk_size = int((avg_chunk_size_pdf_1 + avg_chunk_size_pdf_2) / 2)
chunk_overlap = int(chunk_size * 0.1)  # Set overlap as 10% of chunk_size

In [17]:
# Print out the dynamically chosen chunk size and overlap
print(f"Dynamic chunk_size: {chunk_size}, chunk_overlap: {chunk_overlap}")

Dynamic chunk_size: 46, chunk_overlap: 4


In [18]:
# Step 3: Split the text into smaller chunks using the dynamic chunk_size and chunk_overlap
pdf_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=18)

pdf_1_chunks = pdf_splitter.create_documents([vidhuraneethi_pdf_text])
pdf_2_chunks = pdf_splitter.create_documents([yakshaprasna_pdf_text])

In [19]:
pdf_1_chunks

[Document(page_content='Vidura Niti  \n॥  ॥'),
 Document(page_content='॥  ॥ \n \n \n \n \n \n \nSource of E -texts:'),
 Document(page_content='Sanskrit UnicodeText:'),
 Document(page_content='UnicodeText: www.sanskritdocuments.org'),
 Document(page_content='Sanskrit Text of Mahabharata: Vedic Reserve'),
 Document(page_content='Vedic Reserve at'),
 Document(page_content='www.mum.org'),
 Document(page_content='English Translation by KM Ganguli:'),
 Document(page_content='www.sacred -texts.com  Vidura Niti  \n2 ॥अथ'),
 Document(page_content='2 ॥अथ  \n॥'),
 Document(page_content='॥ \n(Vidura’s teachings on right conduct'),
 Document(page_content='extracted from Mahabharata)  \n \n। \n  \n॥ १॥'),
 Document(page_content='"Vaisampayana said, \'King Dhritarashtra'),
 Document(page_content='endued with great wisdom (then) said to the'),
 Document(page_content="orderly -in-waiting, 'I desire to see Vidura."),
 Document(page_content='to see Vidura.'),
 Document(page_content="Bring him here withou

In [20]:
print(type(pdf_1_chunks))

<class 'list'>


In [21]:
print(len(pdf_1_chunks))

3292


In [22]:
print(len(pdf_2_chunks))

1041


In [23]:
pdf_2_chunks

[Document(page_content='Yaksha Prashna\nयǘ ĲŬन'),
 Document(page_content='यǘ ĲŬन\nThe Story of the Righteous Crane'),
 Document(page_content='Mahabharata, Aranya Parva , 311–12'),
 Document(page_content='Vaisampayana said, ‘Yudhishthira saw his'),
 Document(page_content='saw his brothers, each possessed of the glory'),
 Document(page_content='of the glory of'),
 Document(page_content='Indra himself, lying dead like the Regents of'),
 Document(page_content='the Regents of the world dropped from their'),
 Document(page_content='from their spheres'),
 Document(page_content='at the end of the Yuga. And beholding Arjuna'),
 Document(page_content='beholding Arjuna lying dead, with his bow and'),
 Document(page_content='with his bow and arrows'),
 Document(page_content='dropped on the ground, and also Bhimasena and'),
 Document(page_content='Bhimasena and the twins motionless and'),
 Document(page_content='motionless and deprived'),
 Document(page_content='of life, the king breathed a hot and

In [24]:
print(pdf_1_chunks[5])

page_content='Vedic Reserve at'


In [25]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")

In [26]:
final_pdf_1_chunks =pdf_1_chunks+pdf_2_chunks

In [27]:
print(len(final_pdf_1_chunks))

4333


In [28]:
print(type(embeddings))

<class 'langchain_community.embeddings.ollama.OllamaEmbeddings'>


In [29]:
def embed_with_progress_bar(embeddings, documents):
        """Embeds documents with a tqdm progress bar (synchronous)."""
        embeddings_list = []
        for doc in tqdm(documents, desc="Embedding Documents"):
                try:
                        embedding = embeddings.embed_query(doc)
                        embeddings_list.append(embedding)
                except Exception as e:
                        print(f"Error embedding text: {doc[:50]}... Error: {e}")
        return embeddings_list

In [30]:
print(type(final_pdf_1_chunks))

<class 'list'>


In [31]:
print(final_pdf_1_chunks)

[Document(page_content='Vidura Niti  \n॥  ॥'), Document(page_content='॥  ॥ \n \n \n \n \n \n \nSource of E -texts:'), Document(page_content='Sanskrit UnicodeText:'), Document(page_content='UnicodeText: www.sanskritdocuments.org'), Document(page_content='Sanskrit Text of Mahabharata: Vedic Reserve'), Document(page_content='Vedic Reserve at'), Document(page_content='www.mum.org'), Document(page_content='English Translation by KM Ganguli:'), Document(page_content='www.sacred -texts.com  Vidura Niti  \n2 ॥अथ'), Document(page_content='2 ॥अथ  \n॥'), Document(page_content='॥ \n(Vidura’s teachings on right conduct'), Document(page_content='extracted from Mahabharata)  \n \n। \n  \n॥ १॥'), Document(page_content='"Vaisampayana said, \'King Dhritarashtra'), Document(page_content='endued with great wisdom (then) said to the'), Document(page_content="orderly -in-waiting, 'I desire to see Vidura."), Document(page_content='to see Vidura.'), Document(page_content="Bring him here without  delay.'  \n \

In [32]:
# # Create an empty list to store the embeddings
# embeddings_list = []

# # Initialize the tqdm progress bar to track embedding progress
# for doc in tqdm(final_pdf_1_chunks, desc="Embedding Documents", unit="doc"):
#     # Embed each document and append it to the embeddings_list
#     embedding = embeddings.embed_query(doc)
#     embeddings_list.append(embedding)

# # Convert embeddings_list to a numpy array (required for FAISS)
# embeddings_array = np.array(embeddings_list)

# # # Create the FAISS index (using L2 distance)
# # index = faiss.IndexFlatL2(embeddings_array.shape[1])  # L2 index type
# # index.add(embeddings_array)  # Add the embeddings to the FAISS index

# # # The index is now ready to be used for similarity search or other tasks
# # print("FAISS index creation complete!")

Embedding Documents:   0%|          | 0/4333 [00:00<?, ?doc/s]
Embedding Documents: 100%|██████████| 4333/4333 [3:16:22<00:00,  2.72s/doc]  

In [33]:
# print(embeddings_array.shape)

In [34]:
# print(embeddings_array)

In [35]:
# Saving the embedding to a file
# np.save('embeddings.npy', np.array(embeddings_array))

In [36]:
# Load the embeddings from the saved file
embeddings_array = np.load('embeddings.npy')

In [37]:
print(embeddings_array.shape)

(4333, 1024)


In [38]:
# Getting the data from the .env file
load_dotenv(dotenv_path=r"G:\My Drive\GenAI api keys\.env")
#print(os.environ.get("PINE_CONE_API_KEY"))
# print(os.environ.get("GROQ_API_KEY"))

True

In [39]:
# Prepare data in Pinecone's required format (ID, embedding, metadata)
pinecone_data=[]

## initilize the pine cone